In [ ]:
#About the data:
#https://www.kaggle.com/datasets/kamilpytlak/personal-key-indicators-of-heart-disease

# State	                    : 거주구역
# Sex	                    : 성별
# GeneralHealth	            : 일상적인 몸 상태 [Excellent, Very good, Good, Fair, Poor]
# PhysicalHealthDays	    : 30일 기준, 질병이나 부상 등 신체적으로 아픈 날 수
# MentalHealthDays	        : 30일 기준, 우울이나 스트레스 등 정신적으로 아픈 날 수
# LastCheckupTime	        : 마지막으로 검진 받은 기록 [1년 이내, 1~2년, 2~5년, 5년 이상]
# PhysicalActivities	    : 운동 / 신체적 활동 있는지
# SleepHours	            : 수면 시간
# RemovedTeeth	            : 발치 수
# HadHeartAttack	        : 심장마비 경험 //심장
# HadAngina                 : 협심증 //심장
# HadStroke                 : 뇌졸증 //뇌
# HadAsthma                 : 천식 //폐
# HadSkinCancer             : 피부암
# HadCOPD                   : 만성폐쇄성폐질환 //폐
# HadDepressiveDisorder     : 우울증 
# HadKidneyDisease          : 신부증
# HadArthritis              : 관절염
# HadDiabetes               : 당뇨 [당뇨 아님, 전당뇨거나 경계성 당뇨, 임신성 당뇨, 당뇨]
# DeafOrHardOfHearing       : 청각장애
# BlindOrVisionDifficulty	: 시각장애
# DifficultyConcentrating	: 집중하기 힘듦
# DifficultyWalking	        : 걷기 힘듦
# DifficultyDressingBathing	: 스스로 옷을 입거나 씻기 힘듦
# DifficultyErrands	        : 일상 업무에 어려움
# SmokerStatus	            : 흡연자 [비흡연자, 금연, 종종 흡연, 매일 흡연]
# ECigaretteUsage	        : 전자담배 흡연자 [경헙 없음, 현재는 사용하지 않음, 종종 흡연, 매일 흡연]
# ChestScan	                : 가슴 CT 나 X-ray 촬영 여부
# RaceEthnicityCategory	    : 인종 [히스패닉, 흑인, 백인, 여러인종, 다른 인종) [다른 인종들은 히스패닉이 아닌 경우에만 표시]
# AgeCategory	            : 나이 카테고리 [18~24, 25~29, 30~34, 35~39, 40~44, 45~49, 50~54, 55~59, 60~64, 65~69, 70~74, 75~79, 80이상]
# HeightInMeters	        : 키
# WeightInKilograms	        : 몸무게
# BMI	                    : BMI
# AlcoholDrinkers	        : 음주 여부
# HIVTesting	            : HIV 검사 결과
# FluVaxLast12	            : 12개월 안에 감기 백신 접종 여부
# PneumoVaxEver	            : 폐렴 백신 접종 경험
# TetanusLast10Tdap	        : 10년 안에 파상풍 주사 경험 [경험 없음, Tdap 주사, Tdap 외의 다른 약물 주사, 어떤 약물인지 모름]
# HighRiskLastYear	        : 지난해 위험한 사고나 질병 경험 유무
# CovidPos                  : 코로나 확진 여부

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.preprocessing import RobustScaler, StandardScaler, MaxAbsScaler
import lightgbm as lgb
from sklearn.ensemble import RandomForestClassifier
from collections import Counter
from sklearn.svm import SVC
from sklearn.pipeline import make_pipeline

데이터 불러오기

In [3]:
#Work with new data

new_data = pd.read_csv("Heart_AfterPrep_0228.csv")
new_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 274034 entries, 0 to 274033
Data columns (total 40 columns):
 #   Column                     Non-Null Count   Dtype  
---  ------                     --------------   -----  
 0   State                      274034 non-null  object 
 1   Sex                        274034 non-null  object 
 2   GeneralHealth              274034 non-null  object 
 3   PhysicalHealthDays         274034 non-null  float64
 4   MentalHealthDays           274034 non-null  float64
 5   LastCheckupTime            274034 non-null  object 
 6   PhysicalActivities         274034 non-null  object 
 7   SleepHours                 274034 non-null  float64
 8   RemovedTeeth               274034 non-null  object 
 9   HadHeartAttack             274034 non-null  object 
 10  HadAngina                  274034 non-null  object 
 11  HadStroke                  274034 non-null  object 
 12  HadAsthma                  274034 non-null  object 
 13  HadSkinCancer              27

In [4]:
#일단 임시로 파일 복사후, 그 파일로 작업해보기

temp = new_data.copy()

temp["HeartDisease"] = 0
temp.loc[temp["HadHeartAttack"] == "Yes","HeartDisease"] = 1
temp.loc[temp["HadAngina"] == "Yes","HeartDisease"] = 1

temp["LungDisease"] = 0
temp.loc[temp["HadAsthma"] == "Yes", "LungDisease"] = 1
temp.loc[temp["HadCOPD"] == "Yes", "LungDisease"] = 1

# label encoding 하기 전에, 몇몇 column 들은 0~5 같은 숫자로 (poor, fair, good, very good, excellent 같은) 변환 가능한것들이 있어서 그것들을 먼저 변환 해주기.\
# label encoding 으로 할시에 이것들이 원하는식으로 변환이 안됨.



In [5]:

#General Health encoding
temp.loc[temp["GeneralHealth"] == "Poor","GeneralHealth"] = 0
temp.loc[temp["GeneralHealth"] == "Fair","GeneralHealth"] = 1
temp.loc[temp["GeneralHealth"] == "Good","GeneralHealth"] = 2
temp.loc[temp["GeneralHealth"] == "Very good","GeneralHealth"] = 3
temp.loc[temp["GeneralHealth"] == "Excellent","GeneralHealth"] = 4
temp["GeneralHealth"] = temp["GeneralHealth"].astype('int32')

#Last check up time encoding
temp.loc[temp["LastCheckupTime"] == "Within past year (anytime less than 12 months ago)","LastCheckupTime"] = 0
temp.loc[temp["LastCheckupTime"] == "Within past 2 years (1 year but less than 2 years ago)","LastCheckupTime"] = 1
temp.loc[temp["LastCheckupTime"] == "Within past 5 years (2 years but less than 5 years ago)","LastCheckupTime"] = 2
temp.loc[temp["LastCheckupTime"] == "5 or more years ago","LastCheckupTime"] = 3
temp["LastCheckupTime"] = temp["LastCheckupTime"].astype('int32')

#SmokerStatus
temp.loc[temp["SmokerStatus"] == "Never smoked","SmokerStatus"] = 0
temp.loc[temp["SmokerStatus"] == "Former smoker","SmokerStatus"] = 1
temp.loc[temp["SmokerStatus"] == "Current smoker - now smokes some days","SmokerStatus"] = 2
temp.loc[temp["SmokerStatus"] == "Current smoker - now smokes every day","SmokerStatus"] = 3
temp["SmokerStatus"] = temp["SmokerStatus"].astype('int32')


#ECigaretteUsage
temp.loc[temp["ECigaretteUsage"] == "Never used e-cigarettes in my entire life","ECigaretteUsage"] = 0
temp.loc[temp["ECigaretteUsage"] == "Not at all (right now)","ECigaretteUsage"] = 1
temp.loc[temp["ECigaretteUsage"] == "Use them some days","ECigaretteUsage"] = 2
temp.loc[temp["ECigaretteUsage"] == "Use them every day","ECigaretteUsage"] = 3
temp["ECigaretteUsage"] = temp["ECigaretteUsage"].astype('int32')


#RemovedTeeth
temp.loc[temp["RemovedTeeth"] == "None of them","RemovedTeeth"] = 0
temp.loc[temp["RemovedTeeth"] == "None of Them","RemovedTeeth"] = 0
temp.loc[temp["RemovedTeeth"] == "1 to 5","RemovedTeeth"] = 1
temp.loc[temp["RemovedTeeth"] == "6 or more, but not all","RemovedTeeth"] = 2
temp.loc[temp["RemovedTeeth"] == "All","RemovedTeeth"] = 3
temp["RemovedTeeth"] = temp["RemovedTeeth"].astype('int32')

#HadDiabetes
temp.loc[temp["HadDiabetes"] == "No","HadDiabetes"] = 0
temp.loc[temp["HadDiabetes"] == "Yes, but only during pregnancy (female)","HadDiabetes"] = 0
temp.loc[temp["HadDiabetes"] == "No, pre-diabetes or borderline diabetes","HadDiabetes"] = 1
temp.loc[temp["HadDiabetes"] == "Yes","HadDiabetes"] = 2
temp["HadDiabetes"] = temp["HadDiabetes"].astype('int32')

#Covid
temp['CovidPos'].replace({'Tested positive using home test without a health professional' : 'Yes'}, inplace=True)

# 나머지 인코딩

In [6]:
#나이 인코딩
ages = {
    'Age 80 or older': 80,
    'Age 55 to 59': 55,
    'Age 40 to 44': 40,
    'Age 75 to 79': 75,
    'Age 70 to 74': 70,
    'Age 65 to 69': 65,
    'Age 60 to 64': 60,
    'Age 50 to 54': 50,
    'Age 45 to 49': 45,
    'Age 35 to 39': 35,
    'Age 30 to 34': 30,
    'Age 25 to 29': 25,
    'Age 18 to 24': 18
}

temp['AgeCategory'] = temp['AgeCategory'].map(lambda x: ages[x])

In [7]:
#one hot encoding for RaceEthnicityCategory, TetanusLast10Tdap

temp_encoded = pd.get_dummies(temp, columns = ['RaceEthnicityCategory', 'TetanusLast10Tdap'])

for col in temp_encoded.columns:
    print(col)
    print(temp_encoded[col].unique())
    print('-'*50)

State
['Alabama' 'Alaska' 'Arizona' 'Arkansas' 'California' 'Colorado'
 'Connecticut' 'Delaware' 'District of Columbia' 'Florida' 'Georgia'
 'Hawaii' 'Idaho' 'Illinois' 'Indiana' 'Iowa' 'Kansas' 'Kentucky'
 'Louisiana' 'Maine' 'Maryland' 'Massachusetts' 'Michigan' 'Minnesota'
 'Mississippi' 'Missouri' 'Montana' 'Nebraska' 'Nevada' 'New Hampshire'
 'New Jersey' 'New Mexico' 'New York' 'North Carolina' 'North Dakota'
 'Ohio' 'Oklahoma' 'Oregon' 'Pennsylvania' 'Rhode Island' 'South Carolina'
 'South Dakota' 'Tennessee' 'Texas' 'Utah' 'Vermont' 'Virginia'
 'Washington' 'West Virginia' 'Wisconsin' 'Wyoming' 'Guam' 'Puerto Rico'
 'Virgin Islands']
--------------------------------------------------
Sex
['Female' 'Male']
--------------------------------------------------
GeneralHealth
[4 1 0 3 2]
--------------------------------------------------
PhysicalHealthDays
[ 0.  2.  1.  5. 30. 23. 14.  4. 15.  3. 10.  7. 25.  6. 21. 20. 29.  8.
 16.  9. 27. 12. 13. 11. 28. 17. 24. 26. 18. 22. 19.]
---

In [8]:
le = LabelEncoder()

for columns in temp_encoded.select_dtypes(include=[object, bool]).columns:
    temp_encoded[columns] = le.fit_transform(temp_encoded[columns]) # 전처리 하기 전에 Object 값을 가지고 있는 행을 일단 일시적으로 label encoding 해보기.
    #No = 0
    #yes = 1
    


# 이상치 처리

In [9]:
#나누기전에 이상치 제거. Height, weight , BMI, sleeping hour 에 비정상적인 데이터들이 보임. 
# 다른 float 데이터인 physical day health, mental day health 는 계속 아프거나 안아플수 있다고 생각함.

arr = []

for cols in temp_encoded[['BMI', 'SleepHours', 'WeightInKilograms', 'HeightInMeters']]:
    Q1 = np.percentile(temp_encoded[cols], 25, method='midpoint')
    Q3 = np.percentile(temp_encoded[cols], 75, method='midpoint')
    IQR = Q3-Q1
    upper = Q3 + 1.5*IQR
    lower = Q1 - 1.5*IQR

    upper_array = temp_encoded.loc[temp_encoded[cols] >= upper].index.values
    lower_array = temp_encoded.loc[temp_encoded[cols] <= lower].index.values

    arr = np.hstack((arr , upper_array , lower_array))


    
# remove the outliers
temp_encoded.drop(index=np.unique(arr), inplace=True)


# Data imbalance 처리

In [11]:
from imblearn.over_sampling import SMOTE

X = temp_encoded.drop(columns=['HeartDisease', "HadHeartAttack", "HadAngina", 'State' ])
y = temp_encoded['HeartDisease']



print('Original Dataset shape %s' % Counter(y))
smo = SMOTE(sampling_strategy = 0.8, random_state = 1234)
X_smo, y_smo = smo.fit_resample(X, y)
print('Resampled Dataset shape %s' % Counter(y_smo))
smoDF = pd.concat([X_smo, y_smo], axis = 1)

smoDF = pd.concat([X_smo, y_smo], axis = 1)

tmp_yes = temp_encoded[temp_encoded['HeartDisease'] == 1]
smo_yes = smoDF[smoDF['HeartDisease'] == 1]


print(tmp_yes.duplicated().sum())
print(smo_yes.duplicated().sum())

Original Dataset shape Counter({0: 234057, 1: 22525})
Resampled Dataset shape Counter({0: 234057, 1: 187245})
1
1848


Neural net 으로 하기에 너무 dimension 이 많아서 computation heavy 할듯 보여서 20개 feature 만 추려서 하기.

In [13]:
train_x = smoDF.drop(columns=['HeartDisease' ])
train_y = pd.array(smoDF['HeartDisease'])

X_train, X_test, y_train, y_test = train_test_split(train_x, train_y, test_size=0.2, random_state=1234)

In [14]:
#Fisher's score 로 feature importance 보기
from sklearn.feature_selection import f_classif

#스코어 계산
# Compute Fisher scores
scores, _ = f_classif(X_train, y_train)

feature_impt = pd.Series(scores, index=X_train.columns)

feature_impt_sorted = feature_impt.sort_values(ascending=False)
print(feature_impt_sorted[:20])

GeneralHealth                                                                  93836.789849
AgeCategory                                                                    74536.228337
AlcoholDrinkers                                                                36150.607968
PhysicalActivities                                                             30432.287364
ChestScan                                                                      22802.564184
TetanusLast10Tdap_Yes, received Tdap                                           21250.969742
CovidPos                                                                       20526.168928
LastCheckupTime                                                                20415.816753
HIVTesting                                                                     19791.892035
PhysicalHealthDays                                                             18555.696847
ECigaretteUsage                                                                1

In [20]:
# feature 숫자 20개만 넣고 돌려보기

train_x = smoDF[feature_impt_sorted[:20].index.to_list()]
train_y = pd.array(smoDF['HeartDisease'])

X_train, X_test, y_train, y_test = train_test_split(train_x, train_y, test_size=0.2, random_state=1234)

# NN 시작

In [21]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

In [43]:
#스케일링 
rbs = RobustScaler()
X_train_scaled = rbs.fit_transform(X_train)
X_test_scaled = rbs.transform(X_test)

print(X_train_scaled.shape)

#tensor 로 변환
X_train_tensor = torch.tensor(X_train_scaled, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test_scaled, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.long)
y_test_tensor = torch.tensor(y_test, dtype=torch.long)

#그것으로 data loader 생성
train_ds = TensorDataset(X_train_tensor, y_train_tensor)
test_ds = TensorDataset(X_test_tensor, y_test_tensor)

(337041, 20)


In [44]:
class HeartDiseaseNet(nn.Module):
    def __init__(self):
        super(HeartDiseaseNet, self).__init__()

        self.fc1 = nn.Linear(20,64)
        self.fc2 = nn.Linear(64,48)
        self.fc3 = nn.Linear(48,32)
        self.fc4 = nn.Linear(32,16)
        self.fc5 = nn.Linear(16,8)
        self.output = nn.Linear(8,2)
        self.act = nn.ReLU()

    def forward(self, x):
        x = self.act(self.fc1(x))
        x = self.act(self.fc2(x))
        x = self.act(self.fc3(x))
        x = self.act(self.fc4(x))
        x = self.act(self.fc5(x))
        x = self.output(x)
        return x



In [45]:
model = HeartDiseaseNet()
loss_fn = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

print(HeartDiseaseNet())

HeartDiseaseNet(
  (fc1): Linear(in_features=20, out_features=64, bias=True)
  (fc2): Linear(in_features=64, out_features=48, bias=True)
  (fc3): Linear(in_features=48, out_features=32, bias=True)
  (fc4): Linear(in_features=32, out_features=16, bias=True)
  (fc5): Linear(in_features=16, out_features=8, bias=True)
  (output): Linear(in_features=8, out_features=2, bias=True)
  (act): ReLU()
)


In [50]:
train_loader = DataLoader(dataset = train_ds, batch_size = 100, shuffle=True)
test_loader = DataLoader(dataset = test_ds, batch_size = 100, shuffle=True)

for epoch in range(1000):
    total_loss = 0.0
    for inputs, labels in train_loader:
        optimizer.zero_grad() # 기울기를 0으로 초기화
        outputs = model(inputs) # 순전파
        loss = loss_fn(outputs, labels) # 손실 계산
        loss.backward() # 역전파
        optimizer.step() # 매개변수 갱신
        total_loss += loss.item()

    if (epoch+1) % 50 == 0:
        print(f'Epoch {epoch+1}, Loss: {total_loss/len(train_loader)}')


Epoch 50, Loss: 0.25962024258333083
Epoch 100, Loss: 0.25742716126053317
Epoch 150, Loss: 0.25560461982832716
Epoch 200, Loss: 0.25467220280740493
Epoch 250, Loss: 0.2538105788781854
Epoch 300, Loss: 0.25326367154235624
Epoch 350, Loss: 0.2522428274857506
Epoch 400, Loss: 0.25188546879501594
Epoch 450, Loss: 0.25141817891599516
Epoch 500, Loss: 0.2511353892384987
Epoch 550, Loss: 0.25053570313018353
Epoch 600, Loss: 0.25048776142226237
Epoch 650, Loss: 0.2499989708363391
Epoch 700, Loss: 0.249909788031972
Epoch 750, Loss: 0.24969722861238985
Epoch 800, Loss: 0.2496537449309678
Epoch 850, Loss: 0.24940399994211146
Epoch 900, Loss: 0.249003392257718
Epoch 950, Loss: 0.24910480390269407
Epoch 1000, Loss: 0.24848564054669983


In [51]:
# 모델 평가 진행
model.eval() # 모델 평가 모드
with torch.no_grad(): # 기울기 계산 없음(평가시에는 모델 업데이트를 하지 않으므로)
    total = 0    # 전체 샘플의 수 초기값 0
    correct = 0  # 올바르게 예측된 샘플수 초기값 0
    for inputs, labels in test_loader:  # 테스트 데이터셋에서 배치 단위로 입력 데이터(inputs)와 정답 레이블(labels) 호출
        outputs = model(inputs) # 현재 배치의 입력 데이터를 모델에 전달, 예측 수행합니다. 클래스에 대한 모델의 예측 확률(점수) 리턴
        _, predicted = torch.max(outputs.data, 1) # torch.max() 함수를 통해 예측 확률이 가장 높은 클래스의 인덱스를 가져옴.
           # torch.max()두 개의 값 리턴. 1 각 행(또는 열)의 최대값 2 : 해당 최대값의 인덱스
           # '_' 변수명... 관계적으로 관심이 없는 값을 받기 위해 사용하는 변수명
           # 1: 최대값을 찾을 차원 지정(각 행에 대해 최대값을 찾도록 지시하는 옵션 값, 열은 0)
           # predicted : 모델이 예측한 클래스의 인덱스
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Accuracy of the network on the test set: %d %%' % (100 * correct / total))

Accuracy of the network on the test set: 86 %


In [53]:
if torch.cuda.is_available():
  X_train_tensor_gpu = X_train_tensor.to('cuda')
  X_test_tensor_gpu = X_test_tensor.to('cuda')
  y_train_tensor_gpu = y_train_tensor.to('cuda')
  y_test_tensor_gpu = y_test_tensor.to('cuda')
  # print(f"Device tensor is stored on: {tensor.device}")



#그것으로 data loader 생성
train_ds_gpu = TensorDataset(X_train_tensor_gpu, y_train_tensor_gpu)
test_ds_gpu = TensorDataset(X_test_tensor_gpu, y_test_tensor_gpu)

In [56]:
model = HeartDiseaseNet()
model.to('cuda')
loss_fn = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

print(HeartDiseaseNet())

HeartDiseaseNet(
  (fc1): Linear(in_features=20, out_features=64, bias=True)
  (fc2): Linear(in_features=64, out_features=48, bias=True)
  (fc3): Linear(in_features=48, out_features=32, bias=True)
  (fc4): Linear(in_features=32, out_features=16, bias=True)
  (fc5): Linear(in_features=16, out_features=8, bias=True)
  (output): Linear(in_features=8, out_features=2, bias=True)
  (act): ReLU()
)


In [59]:
train_loader_gpu = DataLoader(dataset = train_ds_gpu, batch_size = 100, shuffle=True)
test_loader_gpu = DataLoader(dataset = test_ds_gpu, batch_size = 100, shuffle=True)

for epoch in range(100):
    total_loss = 0.0
    for inputs, labels in train_loader_gpu:
        optimizer.zero_grad() # 기울기를 0으로 초기화
        outputs = model(inputs) # 순전파
        loss = loss_fn(outputs, labels) # 손실 계산
        loss.backward() # 역전파
        optimizer.step() # 매개변수 갱신
        total_loss += loss.item()

    if (epoch+1) % 10 == 0:
        print(f'Epoch {epoch+1}, Loss: {total_loss/len(train_loader_gpu)}')


In [ ]:
# 모델 평가 진행
model.eval() # 모델 평가 모드
with torch.no_grad(): # 기울기 계산 없음(평가시에는 모델 업데이트를 하지 않으므로)
    total = 0    # 전체 샘플의 수 초기값 0
    correct = 0  # 올바르게 예측된 샘플수 초기값 0
    for inputs, labels in test_loader_gpu:  # 테스트 데이터셋에서 배치 단위로 입력 데이터(inputs)와 정답 레이블(labels) 호출
        outputs = model(inputs) # 현재 배치의 입력 데이터를 모델에 전달, 예측 수행합니다. 클래스에 대한 모델의 예측 확률(점수) 리턴
        _, predicted = torch.max(outputs.data, 1) # torch.max() 함수를 통해 예측 확률이 가장 높은 클래스의 인덱스를 가져옴.
           # torch.max()두 개의 값 리턴. 1 각 행(또는 열)의 최대값 2 : 해당 최대값의 인덱스
           # '_' 변수명... 관계적으로 관심이 없는 값을 받기 위해 사용하는 변수명
           # 1: 최대값을 찾을 차원 지정(각 행에 대해 최대값을 찾도록 지시하는 옵션 값, 열은 0)
           # predicted : 모델이 예측한 클래스의 인덱스
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Accuracy of the network on the test set: %d %%' % (100 * correct / total))